지금까지 우리가 다룬 정형 데이터들을 다루는데 가장 뛰어난 성과를 내는 알고리즘은 앙상블 학습(ensemble learning)이다. 이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있다.(반대로 비정형 데이터를 다룰 때 유용한 알고리즘은 신경망 알고리즘이다.)<br/><br/>
__랜덤 포레스트(random forest)__는 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용되고 있다.<br>
이름에서 알 수 있듯이 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.<br/>
랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데 훈련 데이터에서 복원추출하여 샘플을 만드는 부트스트랩 샘플(bootstrap sample) 방식을 이용한다. 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.<br/>
또한 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다. 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다. 다만 회귀 모델인 RandomForestRegressor는 전체 특성을 사용한다.<br/>
랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련한다. 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼고, 회귀일 때는 단순히 각 트리의 예측을 평균한다.<br/><br/>
랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 떄문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다. 종종 기본 매개변수 설정 만으로도 아주 좋은 결과를 낸다.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
from sklearn.model_selection import cross_validate      #교차 검증 수행
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)      #return_train_score : 훈련 세트에 대한 점수 반환 여부.
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)      #특성 중요도 출력
##결정트리로 구한 중요도 [0.12345626 0.86862934 0.0079144 ]와 비교하면, 하나에 특성에 과도하게 집중하지 않게 됐다는것을 알수있다. 이는 일반회 성능을 높이는데 도움이 된다.

[0.23167441 0.50039841 0.26792718]


랜덤 포레스트는 훈련세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리를 훈련하기 때문에 남는 샘플이 생긴다. 이러한 샘플을 OOB(Out of bag)샘플이라고 부르는데 이 OOB 샘플을 이용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가 할 수 있고, 이 점수는 RandomForestClassifier 클래스의 oob_score_ 매개변수에 저장된다.

In [6]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)     #oob_score : OOB 점수 반환 여부
rf.fit(train_input, train_target)
print(rf.oob_score_)
#oob점수를 보면 교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻은 것을 알 수 있다.
##OOB점수를 사용하면 교차검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

0.8934000384837406


__엑스트라 트리(extra Trees)__는 랜덤 포레스트와 매우 비슷하게 동작한다.<br/>
기본적으로 100개의 결정 트리를 훈련하고 랜덤 포레스트와 동일하게 결정트리가 제공하는 대부분의 매개변수를 지원한다. 또한 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용한다.<br/>
랜덤 포레스트와의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점이다. 즉각 결정 트리를 만들 때 전체 훈련 세트를 사용하는 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않다. 보통 엑스트라 트리가 무작위성이 더 크기 때문에 랜덤 포레스트 보다 더 많은 결정 트리를 훈련해야 한다. 하지만 랜덤하게 노드를 분할하기 때문에 계산속도가 빠른것이 장점이다.

In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)      #특성 중요도 출력. 역시 결정 트리보다 당도 의존성이 낮다.

[0.20183568 0.52242907 0.27573525]


__그레이디언트 부스팅(gradient boosting)__은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다.<br/>
사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용한다. 깊이가 얕기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.<br/>
그레이디언트 부스팅은 경사 하강법을 사용하여 트리를 앙상블에 추가한다. 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용한다.

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [10]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)     #n_estimators : 결정트리 개수, learning_rate : 학습률(기본값 0.1)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))      #결정 트리 개수를 5배 늘렸지만 과대적합을 잘 억제했음.

0.9464595437171814 0.8780082549788999


In [11]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)      #특성 중요도 출력. 랜덤 포레스트보다 일부특성(당도)에 더 집중한다.

[0.15872278 0.68010884 0.16116839]


__히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)__은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘이다. 히스토그램 기반 그레이디언트 부스팅은 먼저 입력 특성을 256개의 구간으로 나누기 때문에 노드를 분할 할 때 최적의 분할을 매우 빠르게 찾을 수 있다.<br/>
히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용한다. 따라서 입력에 누락된 특성이 있어도 이를 따로 전처리할 필요가 없다.<br/>
사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스는 HistGradientBoostingClassifier이다. 일반적으로 HistGradientBoostingClassifier는 기본 매개변수에서 안정적인 성능을 얻을 수 있다. 이 클래스는 트리의 개수를 지정하는데 n_estimators 대신에 부스팅 반복 횟수를 지정하는 max_iter를 사용한다.<br/>
사이킷런의 히스토그램 기반 그레이디언트 부스팅은 아직 테스트 과정에 있다. 이 클래스를 사용하려면 sklearn.experimental 패키지 아래에 있는 enable_hist_gradient_boosting 모듈을 임포트 해야 한다.

In [13]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))      #과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공한다.

0.9321723946453317 0.8801241948619236


히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 permutation_importance() 함수를 사용하자. 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다. 훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있다.<br/>
permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)를 담고 있다.

In [15]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)     #n_repeats : 랜덤하게 섞을 횟수를 지정. 기본값은 5
print(result.importances_mean)      #훈련 세트 특성 중요도 출력. 랜덤 포레스트와 비슷한 비율임을 알 수 있다.

[0.08876275 0.23438522 0.08027708]


In [16]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)      #테스트 세트 특성 중요도 출력. 당도에 조금 더 집중하고 있다는 것을 알 수 있다.

[0.05969231 0.20238462 0.049     ]


In [17]:
hgb.score(test_input, test_target)      #테스트 세트 정확도는 약 87% 실전에선 이보다 조금 더 낮을 것으로 기대.

0.8723076923076923

히스토그램 기반 그레이디언트 부스팅의 회귀 버전은 HistGradientBoostingRegressor 클래스에 구현되어 있다. 사이킷런에서 제공하는 히스토그램 기반 그레이디언트 부스팅은 비교적 새로운 기능이다. 하지만 사이킷런 말고도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여럿 있는데, 가장 대표적인 라이브러리는 XGBoost이다.<br/>
XGBoost는 다양한 부스팅 알고리즘을 지원한다. tree_method 매개변수를 'hist'로 지정하면 히스토그램 기반 그레이디언트 부스팅을 사용할 수 있다. 놀랍게도 사이킷런의 cross_validate()함수와 함께 사용할 수도 있다.

In [18]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는 마이크로소프트에서 만든 LightGBM이다. LightGBM은 빠르고 최신 기술을 많이 적용하고 있어 인기가 점점 높아지고 있다.

In [20]:
from lightgbm import  LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
